In [1]:
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import openpyxl
from modules import * #Contain: stride, batch_size, nPosSoftSize, data_preparation(), decoder(), encoder()

2021-08-29 14:12:24.975880: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
_,label_train,_,label_validation,label_validation_=data_preparation()

label_train = np.append(label_train, [np.fliplr(x) for x in label_train], axis=0)
label_train = np.append(label_train, [np.flipud(x) for x in label_train], axis=0)

print(label_train.shape)
print(label_validation.shape)
print(label_validation_.shape)

(980, 128, 128, 1)
(147, 128, 128, 1)
(3, 512, 512, 1)


In [3]:
train_datagen = ImageDataGenerator()
train_generator = train_datagen.flow(label_train,
                                     label_train,
                                     batch_size = batch_size) 

In [4]:
decoder = decoder()
decoder.compile(optimizer = 'adam', 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])
decoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 1)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 64, 64, 16)        464       
_________________________________________________________________
sequential_1 (Sequential)    (None, 32, 32, 32)        12928     
_________________________________________________________________
sequential_2 (Sequential)    (None, 16, 16, 32)        25728     
_________________________________________________________________
sequential_3 (Sequential)    (None, 8, 8, 32)          25728     
_________________________________________________________________
sequential_4 (Sequential)    (None, 16, 16, 32)        25728     
_________________________________________________________________
sequential_5 (Sequential)    (None, 32, 32, 32)        25728 

2021-08-29 14:12:28.472072: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-08-29 14:12:28.472513: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-08-29 14:12:28.506029: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-29 14:12:28.506335: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2060 SUPER computeCapability: 7.5
coreClock: 1.68GHz coreCount: 34 deviceMemorySize: 7.77GiB deviceMemoryBandwidth: 417.29GiB/s
2021-08-29 14:12:28.506348: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-08-29 14:12:28.507463: I tensorflow/stream_executor/platform/defa

In [5]:
def METRICS(pre_val,label_val):
    """
    Performance evaluation after every epoch
    nPosSoftSize: Buffer size
    tp: True-positive pixels
    fp: False-positive pixels
    fn: False-negative pixels
    """
    global nPosSoftSize
    kernel = np.ones([nPosSoftSize*2+1,nPosSoftSize*2+1])
    def metrics(res,label):
        res=res[:,:,0]
        label=label[:,:,0]
        
        res_dilate=cv2.dilate(res,kernel,1)
        label_dilate=cv2.dilate(label,kernel,1)

        ones_like = tf.ones_like(res)
        zeros_like = tf.zeros_like(res)

        tp = tf.cast(tf.logical_and(tf.equal(res, ones_like),tf.equal(label_dilate, ones_like)),"float")
        fp = tf.cast(tf.logical_and(tf.equal(res, ones_like),tf.equal(label_dilate, zeros_like)),"float")
        precision=tf.reduce_sum(tp)/(tf.reduce_sum(tp)+tf.reduce_sum(fp)+1e-20)

        fn = tf.cast(tf.logical_and(tf.equal(res_dilate, zeros_like),tf.equal(label, ones_like)),"float") 
        recall=tf.reduce_sum(tp)/(tf.reduce_sum(tp)+tf.reduce_sum(fn)+1e-20)
        F1=2*precision*recall/(precision+recall+1e-20)

        precision=np.array(precision)
        recall=np.array(recall)
        F1=np.array(F1)
        return precision,recall,F1
    
    precision=0
    recall=0
    F1=0
    for i in range(pre_val.shape[0]):
        A,B,C=metrics(pre_val[i],label_val[i])
        precision=precision+A
        recall=recall+B
        F1=F1+C
    precision=precision/pre_val.shape[0]
    recall=recall/pre_val.shape[0]
    F1=F1/pre_val.shape[0]
    return precision,recall,F1

In [6]:
EPOCH=1
F_M=0
workbook = openpyxl.Workbook()
worksheet = workbook.create_sheet(index=0)
class MyCustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        '''
        EPOCH: Current epoch
        F_M: Maximum F1-score
        stride: Stride of the sliding window
        H,W: Height and witdh of the original images
        h,w: Height and witdh of the network inputs
        '''
        global EPOCH
        global F_M
        global stride
        pre_validation=np.round(decoder.predict(label_validation))
        pre_validation_=np.zeros(label_validation_.shape)
        H = label_validation_.shape[1]
        W = label_validation_.shape[2]
        h = 128
        w = 128
        stride = stride  
        #Assemble perdiction 
        for i in range(pre_validation_.shape[0]):
            for k1 in range(int((H-h)/ stride)+1):
                for k2 in range(int((W-w)/ stride)+1):
                    pre_validation_[i,k1*stride:k1*stride+h,k2*stride:k2*stride+w]+=pre_validation[i*(int((H-h)/ stride)+1)*(int((W-w)/ stride)+1)+k1*(int((W-w)/ stride)+1)+k2] 
        pre_validation_=np.sign(pre_validation_)  
        precision,recall,F1=METRICS(pre_validation_,label_validation_)

        print( '\n', "precision:", precision, '\n', "recall:", recall, '\n', "F1 :",  F1)

        worksheet.cell(EPOCH,1).value=precision
        worksheet.cell(EPOCH,2).value=recall
        worksheet.cell(EPOCH,3).value=F1
        workbook.save("history/decoder_history.xlsx")

        decoder.save_weights('weights/decoder_weights.h5')
        if F1>F_M:
            F_M=F1
            decoder.save_weights('weights/decoder_weights_best.h5')

        EPOCH=EPOCH+1

In [7]:
history = decoder.fit(
            train_generator,
            epochs = 100,
            verbose = 1,
            callbacks=[MyCustomCallback()])

2021-08-29 02:51:45.483129: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-08-29 02:51:45.500247: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3799900000 Hz


Epoch 1/100


2021-08-29 02:51:45.980707: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


 1/31 [..............................] - ETA: 46s - loss: 0.7577 - accuracy: 0.4253

2021-08-29 02:51:46.951363: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-08-29 02:51:46.953019: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:51:46.967006: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:51:46.979566: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:51:46.991592: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:51:47.003083: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:51:47.015939: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021

...

31/31 [==============================] - 2s 15ms/step - loss: 0.6475 - accuracy: 0.6781

 precision: 0.0 
 recall: 0.0 
 F1 : 0.0
Epoch 2/100
 1/31 [..............................] - ETA: 0s - loss: 0.3852 - accuracy: 0.9401

2021-08-29 02:51:47.684818: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:51:47.695028: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:51:47.702182: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:51:47.709666: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:51:47.718198: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:51:47.725352: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:51:47.737871: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-0

31/31 [==============================] - 0s 12ms/step - loss: 0.3166 - accuracy: 0.9605

 precision: 0.0 
 recall: 0.0 
 F1 : 0.0
Epoch 3/100
31/31 [==============================] - 0s 13ms/step - loss: 0.1429 - accuracy: 0.9847

 precision: 0.0 
 recall: 0.0 
 F1 : 0.0
Epoch 4/100
31/31 [==============================] - 0s 13ms/step - loss: 0.0814 - accuracy: 0.9879

 precision: 0.0 
 recall: 0.0 
 F1 : 0.0
Epoch 5/100
31/31 [==============================] - 0s 12ms/step - loss: 0.0564 - accuracy: 0.9900

 precision: 1.0 
 recall: 0.0032587767927907407 
 F1 : 0.0064838018345957
Epoch 6/100
31/31 [==============================] - 0s 13ms/step - loss: 0.0416 - accuracy: 0.9915

 precision: 1.0 
 recall: 0.019683521551390488 
 F1 : 0.03832160805662473
Epoch 7/100
31/31 [==============================] - 0s 13ms/step - loss: 0.0339 - accuracy: 0.9922

 precision: 1.0 
 recall: 0.16861310849587122 
 F1 : 0.28300513327121735
Epoch 8/100
31/31 [==============================] - 0s 12ms/s

31/31 [==============================] - 0s 11ms/step - loss: 0.0064 - accuracy: 0.9977

 precision: 1.0 
 recall: 0.9993114074071249 
 F1 : 0.9996555050214132
Epoch 51/100
31/31 [==============================] - 0s 11ms/step - loss: 0.0059 - accuracy: 0.9979

 precision: 1.0 
 recall: 0.999538520971934 
 F1 : 0.9997691909472147
Epoch 52/100
31/31 [==============================] - 0s 12ms/step - loss: 0.0061 - accuracy: 0.9978

 precision: 1.0 
 recall: 0.9994282126426697 
 F1 : 0.9997139573097229
Epoch 53/100
31/31 [==============================] - 0s 15ms/step - loss: 0.0058 - accuracy: 0.9979

 precision: 1.0 
 recall: 0.9995703299840292 
 F1 : 0.9997850855191549
Epoch 54/100
31/31 [==============================] - 0s 13ms/step - loss: 0.0058 - accuracy: 0.9979

 precision: 1.0 
 recall: 0.9998187820116679 
 F1 : 0.9999093612035116
Epoch 55/100
31/31 [==============================] - 0s 12ms/step - loss: 0.0057 - accuracy: 0.9979

 precision: 1.0 
 recall: 0.9996327559153239 
 

Epoch 95/100
31/31 [==============================] - 0s 12ms/step - loss: 0.0030 - accuracy: 0.9990

 precision: 0.9997433026631674 
 recall: 0.9995429515838623 
 F1 : 0.999643087387085
Epoch 96/100
31/31 [==============================] - 0s 11ms/step - loss: 0.0028 - accuracy: 0.9991

 precision: 0.9998031655947367 
 recall: 0.9994716246922811 
 F1 : 0.9996373256047567
Epoch 97/100
31/31 [==============================] - 0s 13ms/step - loss: 0.0029 - accuracy: 0.9990

 precision: 0.9993991057078043 
 recall: 0.9998731811841329 
 F1 : 0.9996359944343567
Epoch 98/100
31/31 [==============================] - 0s 13ms/step - loss: 0.0026 - accuracy: 0.9992

 precision: 0.9994838436444601 
 recall: 0.9998805125554403 
 F1 : 0.999682088692983
Epoch 99/100
31/31 [==============================] - 0s 13ms/step - loss: 0.0027 - accuracy: 0.9991

 precision: 0.9995254278182983 
 recall: 0.9992599487304688 
 F1 : 0.9993923107782999
Epoch 100/100
31/31 [==============================] - 0s 13ms